Insper

# Aula 14

Iteradores

In [ ]:
%%writefile stocks.cu
#include <iostream>
#include <fstream>
#include <vector>
#include <cmath>
#include <thrust/host_vector.h>
#include <thrust/device_vector.h>
#include <thrust/reduce.h>
#include <thrust/extrema.h>

using namespace std;

int main() {
    int n =  2518;
    double value = 0.0;
    chrono::duration<double> diff;

    auto leitura_i = chrono::steady_clock::now();
    thrust::host_vector<double> host(n,0);
    for(int i = 0; i<n; i++){
        cin >> value;
        host[i] = value;
    }
    auto leitura_f = chrono::steady_clock::now();
    diff  = leitura_f-leitura_i;
    cout << "Tempo de LEITURA (em segundos)  " << diff.count() << endl;


    auto copia_i = chrono::steady_clock::now();
    thrust::device_vector<double> dev(host);
    auto copia_f = chrono::steady_clock::now();
    diff  = copia_f-copia_i;
    cout << "Tempo de CÓPIA (em segundos)  " << diff.count() << endl;



    // Calcula a média dos preços usando Thrust
    double media = reduce(dev.begin(), dev.end()) / dev.size();

    // Calcula o maior e o menor preço usando Thrust
    auto maxMinPair = thrust::minmax_element(dev.begin(), dev.end());
    double maiorPreco = *maxMinPair.second;
    double menorPreco = *maxMinPair.first;

    // Exibe os resultados
    cout << "Média dos preços nos últimos 10 anos: " << media << endl;

    // Preços no último ano
    thrust::device_vector<double> lastYearPrices(dev.end() - 365, dev.end());
    double lastYearMedia = thrust::reduce(lastYearPrices.begin(), lastYearPrices.end()) / lastYearPrices.size();

    auto lastYearMaxMinPair = thrust::minmax_element(lastYearPrices.begin(), lastYearPrices.end());
    double lastYearMaiorPreco = *lastYearMaxMinPair.second;
    double lastYearMenorPreco = *lastYearMaxMinPair.first;

    cout << "Média dos preços no último ano: " << lastYearMedia << endl;
    cout << "Maior preço dos últimos 10 anos: " << maiorPreco << endl;
    cout << "Menor preço dos últimos 10 anos: " << menorPreco << endl;
    cout << "Maior preço no último ano: " << lastYearMaiorPreco << endl;
    cout << "Menor preço no último ano: " << lastYearMenorPreco << endl;

    return 0;
}


Overwriting stocks.cu


In [ ]:
!nvcc -arch=sm_70 -std=c++14 stocks.cu -o stocks

stocks.cu(15): error: name followed by "::" must be a class or namespace name

stocks.cu(15): error: expected an identifier

stocks.cu(15): error: expected a ";"

stocks.cu(17): error: name followed by "::" must be a class or namespace name

stocks.cu(23): error: name followed by "::" must be a class or namespace name

stocks.cu(24): error: identifier "diff" is undefined

stocks.cu(28): error: name followed by "::" must be a class or namespace name

stocks.cu(30): error: name followed by "::" must be a class or namespace name

8 errors detected in the compilation of "stocks.cu".


In [ ]:
!./stocks < stocks.txt

In [ ]:
%%writefile stocks1.cu
#include<thrust/device_vector.h>
#include <thrust/host_vector.h>
#include <iostream>
#include <chrono>
using namespace std;
int main() {
    int n =  2518;
    double value = 0.0;
    std::chrono::duration<double> diff;

    auto leitura_i = std::chrono::steady_clock::now();
    thrust::host_vector<double> host(n,0);
    for(int i = 0; i<n; i++){
        cin >> value;
        host[i] = value;
    }
    auto leitura_f = std::chrono::steady_clock::now();
    diff  = leitura_f-leitura_i;
    cout << "Tempo de LEITURA (em segundos)  " << diff.count() << endl;


    auto copia_i = std::chrono::steady_clock::now();
    thrust::device_vector<double> dev(host);
    auto copia_f = std::chrono::steady_clock::now();
    diff  = copia_f-copia_i;
    cout << "Tempo de CÓPIA (em segundos)  " << diff.count() << endl;



    // Calcula a média dos preços usando Thrust
    double media = thrust::reduce(dev.begin(), dev.end()) / dev.size();

    // Calcula o maior e o menor preço usando Thrust
    auto maxMinPair = thrust::minmax_element(dev.begin(), dev.end());
    double maiorPreco = *maxMinPair.second;
    double menorPreco = *maxMinPair.first;

    // Exibe os resultados
    cout << "Média dos preços nos últimos 10 anos: " << media << endl;

    // Preços no último ano
    thrust::device_vector<double> lastYearPrices(dev.end() - 365, dev.end());
    double lastYearMedia = thrust::reduce(lastYearPrices.begin(), lastYearPrices.end()) / lastYearPrices.size();

    auto lastYearMaxMinPair = thrust::minmax_element(lastYearPrices.begin(), lastYearPrices.end());
    double lastYearMaiorPreco = *lastYearMaxMinPair.second;
    double lastYearMenorPreco = *lastYearMaxMinPair.first;

    cout << "Média dos preços no último ano: " << lastYearMedia << endl;
    cout << "Maior preço dos últimos 10 anos: " << maiorPreco << endl;
    cout << "Menor preço dos últimos 10 anos: " << menorPreco << endl;
    cout << "Maior preço no último ano: " << lastYearMaiorPreco << endl;
    cout << "Menor preço no último ano: " << lastYearMenorPreco << endl;


}

Overwriting stocks1.cu


In [ ]:
!nvcc -arch=sm_70 -std=c++14 stocks1.cu -o stocks1

In [ ]:
!./stocks1 < stocks.txt

Tempo de LEITURA (em segundos)  0.00317803
Tempo de CÓPIA (em segundos)  0.114104
Média dos preços nos últimos 10 anos: 494.207
Média dos preços no último ano: 959.588
Maior preço dos últimos 10 anos: 1187.56
Menor preço dos últimos 10 anos: 130.044
Maior preço no último ano: 1187.56
Menor preço no último ano: 753.22


In [ ]:
%%writefile stocks2.cu
#include <utility>
#include <vector>
#include <thrust/device_vector.h>
#include <thrust/host_vector.h>
#include <iostream>
#include <chrono>

using namespace std;

int main() {
    int n =  2518;
    double apple, ms = 0.0;
    std::chrono::duration<double> diff;

    // Leitura do dado
    auto leitura_i = std::chrono::steady_clock::now();
    thrust::host_vector<double> H_apple(n);
    thrust::host_vector<double> H_ms(n);
    for(int i = 0; i<n; i++){
        cin >> apple >> ms;
        H_apple[i] = apple;
        H_ms[i] = ms;
    }
    auto leitura_f = std::chrono::steady_clock::now();
    diff  = leitura_f-leitura_i;
    cout << "Tempo de LEITURA (em segundos)  " << diff.count() << endl;

    // Copia o dados para a GPU
    auto copia_i = std::chrono::steady_clock::now();
    thrust::device_vector<double> D_apple = H_apple;
    thrust::device_vector<double> D_ms = H_ms;
    auto copia_f = std::chrono::steady_clock::now();
    diff  = copia_f-copia_i;
    cout << "Tempo de CÓPIA (em segundos)  " << diff.count() << endl;

    // Obtém as diferenças dos preços
    thrust::device_vector<double> diferencas(D_apple.size());
    thrust::transform(D_ms.begin(), D_ms.end(),
                      D_apple.begin(),
                      diferencas.begin(),
                      thrust::minus<double>());

    // Obtém a média das diferenças
    double media = thrust::reduce(diferencas.begin(), diferencas.end()) / diferencas.size();
    cout << "Diferença média de preços  " << media << endl;

    return 0;
}

Overwriting stocks2.cu


In [ ]:
!nvcc -arch=sm_70 -std=c++14 stocks2.cu -o stocks2

In [ ]:
!./stocks2 < stocks2.csv

Tempo de LEITURA (em segundos)  0.00818358
Tempo de CÓPIA (em segundos)  0.185722
Diferença média de preços  39.4165
